In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForMaskedLM
import os
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
#tokenizer

model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
#model


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#Folders to store the tokenizers and the model
#check if the directory exsist or not then create it

model_folder = "/content/drive/MyDrive/Grad_SA/Pre-training_from_checkpoint/tree_bank/models/model"
tokenizer_folder = "/content/drive/MyDrive/Grad_SA/Pre-training_from_checkpoint/tree_bank/models/tokenizer"
isExist = os.path.exists(model_folder)

if not isExist:

  # Create a new directory because it does not exist
  os.makedirs(model_folder)
  print("The new directory is created!")


isExist = os.path.exists(tokenizer_folder)

if not isExist:

  # Create a new directory because it does not exist
  os.makedirs(tokenizer_folder)
  print("The new directory is created!")




In [ ]:
#path of the full sudanese data "in text "
mono_path = "/content/drive/MyDrive/Grad_SA/col_test.txt"

In [ ]:
from transformers import LineByLineTextDataset

# setting up the data
# step1: textdataset reads each line separately, tokenizes and truncates the lines to block_size, Adds special tokens.
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path= mono_path,
    block_size=64,
)
# step2: Data collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

#step3: setting up the trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    #save the model predictions and checkpoints
    output_dir="/content/drive/MyDrive/Grad_SA/tree_bank roberta-more-pre-trained",
    overwrite_output_dir=True,
    save_total_limit=2,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=500,
    seed=1,
    #report the results and logs
    report_to="wandb",
    run_name="Treebank RoBERTa more pre-training using Transfomers on PCM Data",
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
#training
trainer.train()
#saving
trainer.save_model("/content/drive/MyDrive/Grad_SA/tree_bank/roberta-retrained")



In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/drive/MyDrive/Grad_SA/tree_bank/roberta-retrained",
    tokenizer="xlm-roberta-base"
)
fill_mask("الدكتور قال بكرة ماف <mask>")